#Exercise for PySpark
ใช้ Table ที่กำหนดให้ด้านล่างในการเขียน PySpark  
- c360_l0.profile_drm_t_active_profile_customer_journey  
- c360_l0.billing_rt_t_recharge_daily

(Q1) Query ข้อมูลจากถัง c360_l0.profile_drm_t_active_profile_customer_journey และเก็บลงในตัวแปร profile_df โดยมีเงื่อนไขดังนี้  
- ใช้ข้อมูล ณ เดือน กุมภา2022
- ดูแค่ลูกค้า Pre-paid
- ใช้แค่ register_date ,access_method_num ,network_type ,norms_net_revenue ,norms_voice_minute ,norms_mb_gprs_volume
- เปลี่ยนชื่อ Column  
norms_net_revenue --> arpu_thb  
norms_voice_minute --> mou_minute  
norms_mb_gprs_volume --> vou_mb
- กำหนดกลุ่มโดยใช้ rule based ดังนี้  
vou น้อยกว่า 1 GB , mou น้อยกว่า 10 นาที -- > low_usage  
vou มากกว่า 1 GB , mou น้อยกว่า 10 นาที -- > data_user  
vou มากกว่า 1 GB , mou มากกว่า 10 นาที -- > heavy_user
vou น้อยกว่า 1 GB , mou มากกว่า 10 นาที -- > voice_user

In [0]:
%sql
show partitions c360_l0.profile_drm_t_active_profile_customer_journey 

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions  import col,when

In [0]:
df = spark.table('c360_l0.profile_drm_t_active_profile_customer_journey')
df.display()

In [0]:
# vou น้อยกว่า 1 GB , mou น้อยกว่า 10 นาที -- > low_usage
# vou มากกว่า 1 GB , mou น้อยกว่า 10 นาที -- > data_user
# vou มากกว่า 1 GB , mou มากกว่า 10 นาที -- > heavy_user 
# vou น้อยกว่า 1 GB , mou มากกว่า 10 นาที -- > voice_user


# norms_net_revenue --> arpu_thb
# norms_voice_minute --> mou_minute
# norms_mb_gprs_volume --> vou_mb

profile_df = spark.table('c360_l0.profile_drm_t_active_profile_customer_journey').select('register_date' ,'access_method_num' ,'network_type','norms_net_revenue' ,'norms_voice_minute' ,'norms_mb_gprs_volume')\
.filter((col('partition_month') == '202202') & (col("network_type") == '3GPre-paid'))\
.withColumnRenamed('norms_net_revenue','arpu_thb')\
.withColumnRenamed('norms_voice_minute','mou_minute')\
.withColumnRenamed('norms_mb_gprs_volume','vou_mb')\
.withColumn('voice_user',when((col("vou_mb") < 1/1024) & (col("mou_minute") < 10),'low_usage')\
                        .when((col("vou_mb") > 1/1024) & (col("mou_minute") < 10),'data_user')\
                        .when((col("vou_mb") > 1/1024) & (col("mou_minute") > 10),'heavy_user')\
                        .when((col("vou_mb") < 1/1024) & (col("mou_minute") > 10),'voice_user')\
                        .otherwise('other'))


(Q2) Query ข้อมูลจากถัง c360_l0.billing_rt_t_recharge_daily และเก็บลงในตัวแปร top_up_df โดยมีเงื่อนไขดังนี้  
- ใช้ข้อมูล ณ เดือน กุมภา2022
- ต้องการ column ดังนี้  
sum_face_value จำนวนเงินที่ Top up ทั้งหมด  
avg_face_value จำนวนเงินที่ Top up เฉลี่ยต่อครั้ง  
no_recharge_date จำนวนวันที่เติมเงินทั้งหมด

ปล. face_value คือจำนวนเงินที่เติมเงินต่อครั้ง ,recharge_date คือวันที่ลูกค้าเติมเงิน

In [0]:
spark.table('c360_l0.billing_rt_t_recharge_daily').display()

In [0]:
top_up_df  = spark.table('c360_l0.billing_rt_t_recharge_daily').filter(col("partition_date").between ('20220201','20220228')).groupBy('access_method_num','register_date')\
.agg(F.sum("face_value").alias('sum_face_value')\
     ,F.avg("face_value").alias('avg_face_value')\
    ,F.countDistinct("recharge_date").alias('no_recharge_date'))

# .select('register_date' ,'access_method_num' ,'network_type','norms_net_revenue' ,'norms_voice_minute' ,'norms_mb_gprs_volume')

(Q3) นำ profile_df และ top_up_df ที่ได้จากข้อก่อนมา inner join พร้อมทั้งเขี่ยน script ในการเช็ค Duplicated Row และนับจำนวนคนที่มีทั้งหมด

In [0]:
# joined_df = profile_df.join(top_up_df,(profile_df.access_method_num == top_up_df.access_method_num) &
#                                        (profile_df.register_date == top_up_df.register_date),'left')
                            
joined_df = profile_df.join(top_up_df,['access_method_num','register_date'],'left')
                            
                                            

In [0]:
joined_df.display()

In [0]:
joined_df.groupBy(['access_method_num','register_date'])\
    .count()\
    .where(col('count') > 1)\
    .select(F.sum('count'))\
    .show()

In [0]:
joined_df.count()